In [129]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
from scipy.spatial import distance
import subprocess

folder = r'D:\datos_GPetersii\datos_GPetersii\Fish1\Object\raw\50fps'
os.chdir(folder)
files = sorted(glob.glob('*clean.h5'))
videos = sorted(glob.glob('*.avi'))
print('hay ' + str(len(files)) + ' archivos')

hay 52 archivos


In [152]:
def get_centroids(track, n_keypoints):

    centroids = []

    bodyparts = np.array([x for x in track.columns.get_level_values(1)])
    xpositions = pd.DataFrame(np.zeros((len(track),n_keypoints)), columns=np.unique(bodyparts))
    for i in range(n_keypoints):
        xpositions[np.unique(bodyparts)[i]] = (track[track.columns.get_level_values(0)[0], np.unique(bodyparts)[i], 'x'])
        
    median_xposition = np.median(xpositions, axis=1) #xpos

    ypositions = pd.DataFrame(np.zeros((len(track),n_keypoints)), columns=np.unique(bodyparts))
    for i in range(n_keypoints):
        ypositions[np.unique(bodyparts)[i]] = (track[track.columns.get_level_values(0)[0], np.unique(bodyparts)[i], 'y'])
        
    median_yposition = np.median(ypositions, axis=1) #ypos

    for j in range(len(median_xposition)):
        centroids.append([median_xposition[j], median_yposition[j]])

    return centroids

def calculate_velocity(x, sf, pix_to_cm):
    desplazamiento = [abs(distance.euclidean(x,y)/pix_to_cm) for x, y in zip(x[1:], x[:-1])]
    dt = 1 / sf
    v = [i/dt for i in desplazamiento]
    return v
def save_moment(file, start, end, track, i):
    output_video = file[:24] + '_moment_' + str(i) + '.h5'
    start_frame = start - 100 # Start frame index
    end_frame = end + 100   # End frame index
    k = 'moment_' +str(i)
    moment_df = track.iloc[start_frame:end_frame+1, :]
    moment_df.to_hdf(output_video, key=k)

def extract_frames(input_video, output_video, start_frame, end_frame, fps):
    # Construct ffmpeg command

    ffmpeg_cmd = [
        r'C:\ffmpeg\bin', 
        '-i', input_video,
        '-vf', f'select=\'between(n\,{start_frame}\,{end_frame})\',setpts=PTS-STARTPTS',
        '-r', str(fps),
        output_video
    ]
    
    # Execute ffmpeg command
    print(ffmpeg_cmd)
    subprocess.run(ffmpeg_cmd)


In [155]:
for file, vid in zip(files, videos):
    os.chdir(folder)
    track = pd.read_hdf(file)
    centroids = get_centroids(track, n_keypoints=6)
    velocity = calculate_velocity(centroids, sf=150, pix_to_cm=12)
    vel_per_frame = pd.DataFrame(zip([round(x[0]/12) for x in centroids], [round(y[1]/12) for y in centroids], velocity), columns=['x', 'y', 'v'])
    idx_movement = [i for i,x in enumerate(velocity) if x > 10]
    duration_moments = [i-j for i,j in zip(idx_movement[1:], idx_movement[:-1])]
    single_moments = [1 if i < 50 else 0 for i in duration_moments] 

    start = [0]
    end = []
    for i, x in enumerate(single_moments[:-1]):
        if (x == 1 and single_moments[i-1] == 0):
            start.append(i)
        if (x == 1 and single_moments[i+1] == 0):
            end.append(i+1)
    frames_moments = [(idx_movement[x],idx_movement[y]) for x,y in zip(start, end) if not x==y]
    #os.chdir(folder + '\moments')
    print(os.getcwd())
    for i, moment in enumerate(frames_moments):
        #save_moment(file, start=moment[0], end=moment[1], track=track)
        output_vid =  folder + "\moments\\" + vid[:24] + '_moment_' + str(i) + '.mp4'
        extract_frames(folder + "\\" + vid, output_vid, start_frame=moment[0], end_frame=moment[1], fps=50)
    

D:\datos_GPetersii\datos_GPetersii\Fish1\Object\raw\50fps
['C:\\ffmpeg\\bin', '-i', 'D:\\datos_GPetersii\\datos_GPetersii\\Fish1\\Object\\raw\\50fps\\video2023-06-09T16_46_17_50fps.avi', '-vf', "select='between(n\\,1\\,17815)',setpts=PTS-STARTPTS", '-r', '50', 'D:\\datos_GPetersii\\datos_GPetersii\\Fish1\\Object\\raw\\50fps\\moments\\video2023-06-09T16_46_17_moment_0.mp4']


PermissionError: [WinError 5] Acceso denegado

La siguiente celda plotea la trayectoria para un momento dado. Con el tamano del punto proporcional a la velocidad del pez en ese punto.

In [75]:
import seaborn as sns
def plot_trayectories(vel_per_frame, start_frame, end_frame):
    data = vel_per_frame.iloc[start_frame:end_frame+1, :]
    #graficamos
    fig, ax = plt.subplots()
    points = sns.scatterplot(
        data=data,
        x='x',
        y='y',
        size='v',
        hue='v'
    )
    ax.plot(data['x'], data['y'], c='pink', linewidth=.5, linestyle='--', alpha=.5)
    ax.scatter(x =  439.2/12, y = 207.53/12, c='crimson', s=150, marker='*')
    ax.set_ylim([0,63])
    ax.set_xlim([0,63])
    plt.show()

In [135]:
from random import sample
rand_frames_moments = sample(frames_moments, 5)
rand_frames_moments

[(18561, 19609), (29131, 32429), (1, 17815), (55049, 55130), (36707, 40577)]

In [ ]:
for moment in rand_frames_moments:
    plot_trayectories(vel_per_frame, start_frame=moment[0] - 500, end_frame=moment[1] + 500)

Las siguientes celdas generan videos de muestra de las frames seleccionadas. 

In [143]:
import subprocess

def extract_frames(input_video, output_video, start_frame, end_frame, fps):
    # Construct ffmpeg command

    ffmpeg_cmd = [
        r'C:\ffmpeg', 
        '-i', input_video,
        '-vf', f'select=\'between(n\,{start_frame}\,{end_frame})\',setpts=PTS-STARTPTS',
        '-r', str(fps),
        output_video
    ]
    
    # Execute ffmpeg command
    subprocess.run(ffmpeg_cmd)

In [77]:
vid_folder = 'D:\datos_GPetersii\datos_GPetersii\Fish7\Object\raw'
videos = sorted(glob.glob(vid_folder+'*.avi'))

In [136]:
#vid_folder = '/Volumes/Expansion/datos_GPetersii/datos_GPetersii/Fish1/Object/raw/'
#videos = glob.glob(vid_folder+'*.avi')
#for vid in videos :
input_video = videos[0]
for i, moment in enumerate(rand_frames_moments):
    output_video = input_video[74:-9] + '_moment_' + str(i) + '.avi'
    start_frame = moment[0] - 100 # Start frame index
    end_frame = moment[1] + 100   # End frame index
    fps = 50           # Desired frames per second for output video

    extract_frames(input_video, output_video, start_frame, end_frame, fps)


FileNotFoundError: [WinError 2] El sistema no puede encontrar el archivo especificado